In [1]:
import pandas as pd
from datetime import datetime
#read in original dataframes for cleaning

Budget_o = pd.read_csv("Budget_DF_original.csv", index_col=[0])
Opus_o = pd.read_csv("OpusData_MovieData_utf-8.csv")


In [2]:
Budget_o.head()

,Release Date,Movie,Budget,Domestic Gross,Worldwide Gross
1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,789,705,275"
2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
3,"Apr 26, 2019",Avengers: Endgame,"$400,000,000","$856,706,929","$2,793,009,730"
4,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$65,609,445","$246,300,941"
5,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"


In [3]:
Opus_o.head()

,movie_name,production_year,movie_odid,production_budget,domestic_box_office,international_box_office,rating,creative_type,source,production_method,genre,sequel,running_time
0,Madea's Family Reunion,2006,8220100,10000000,63257940,62581,PG-13,Contemporary Fiction,Based on Play,Live Action,Comedy,1.0,NaN
1,Krrish,2006,58540100,10000000,1430721,31000000,Not Rated,Science Fiction,Original Screenplay,Live Action,Action,1.0,NaN
2,End of the Spear,2006,34620100,10000000,11748661,175380,PG-13,Historical Fiction,Original Screenplay,Live Action,Drama,0.0,NaN
3,A Prairie Home Companion,2006,24910100,10000000,20342852,6373339,PG-13,Contemporary Fiction,Original Screenplay,Live Action,Comedy,0.0,105.0
4,Saw III,2006,5840100,10000000,80238724,83638091,R,Contemporary Fiction,Original Screenplay,Live Action,Horror,1.0,NaN


# Cleaing Opus Data

In [4]:
Opus_o.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1936 entries, 0 to 1935
Data columns (total 13 columns):
movie_name                  1936 non-null object
production_year             1936 non-null int64
movie_odid                  1936 non-null int64
production_budget           1936 non-null int64
domestic_box_office         1936 non-null int64
international_box_office    1936 non-null int64
rating                      1913 non-null object
creative_type               1923 non-null object
source                      1915 non-null object
production_method           1925 non-null object
genre                       1926 non-null object
sequel                      1934 non-null float64
running_time                1822 non-null float64
dtypes: float64(2), int64(5), object(6)
memory usage: 196.7+ KB


In [5]:
#Make a new datafram including just the title and the genre which is all I need
Opus_genre = Opus_o[['movie_name','genre']]

In [6]:
#Quick check for any null values in genre column
Opus_genre.genre.isnull().values.any()

True

In [7]:
#Check how many are null values to see if its reasonable to drop
Opus_genre.genre.isna().sum() 

10

In [8]:
#Drop null values since they onyl .005 of total data set
Opus_clean = Opus_genre.dropna()

In [9]:
#Check to make sure theyre gone cause i be paranoid
Opus_clean.genre.isna().sum() 

0

In [ ]:
#save clean data to file
Opus_clean.to_csv('OpusGenre_clean.csv')

# Cleaning Budget Dataset

In [10]:
Budget_o.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2700 entries, 1 to 100
Data columns (total 5 columns):
Release Date       2700 non-null object
Movie              2700 non-null object
Budget             2700 non-null object
Domestic Gross     2700 non-null object
Worldwide Gross    2700 non-null object
dtypes: object(5)
memory usage: 126.6+ KB


In [11]:
#Make new  year column from last 4 of Release Date
Budget_Year = Budget_o
Budget_Year['Year'] = Budget_o['Release Date'].apply(lambda x: x[-4::])
Budget_Year.head()
#filter df to only include movies 2000 forward
Budget_Year['Year'] = Budget_Year['Year'].astype(int)
# Budget_Year.info()
Budget_2000 = Budget_Year[Budget_Year['Year'] > 2000]
Budget_2000.shape
#Change release data to week number 
Budget_2000['Release Date'] =  pd.to_datetime(Budget_2000['Release Date'],format='%b %d, %Y')
Budget_2000['Week'] = Budget_2000['Release Date'].dt.week

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [12]:
Budget_2000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2046 entries, 1 to 99
Data columns (total 7 columns):
Release Date       2046 non-null datetime64[ns]
Movie              2046 non-null object
Budget             2046 non-null object
Domestic Gross     2046 non-null object
Worldwide Gross    2046 non-null object
Year               2046 non-null int64
Week               2046 non-null int64
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 127.9+ KB


In [14]:
Budget_2000.head(1)

,Release Date,Movie,Budget,Domestic Gross,Worldwide Gross,Year,Week
1,2009-12-18,Avatar,"$425,000,000","$760,507,625","$2,789,705,275",2009,51


#### Turning Numbers as String into Integers

In [16]:
#Clearning $ and, out of the money
columns = ['Worldwide Gross', 'Domestic Gross', 'Budget']
for col in columns:
    Budget_2000[col] = Budget_2000[col].astype(str).apply(lambda x: x.replace(',',''))
    Budget_2000[col] = Budget_2000[col].astype(str).apply(lambda x: x.replace('$',''))
    Budget_2000[col] = Budget_2000[col].astype(int)
    
    


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

In [17]:
#Creat Two new Columns Domestic Profits and Worldwide Profits
Budget_2000['Dom_Profits'] = Budget_2000['Domestic Gross'] - Budget_2000['Budget']
Budget_2000['WW_Profits'] = Budget_2000['Worldwide Gross'] - Budget_2000['Budget']

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
Budget_2000.head(1)

# Cleaning Messy Characters in Budget

In [19]:
Budget_2000['Movie']  = Budget_2000['Movie'].astype(str).apply(lambda x: x.replace("â" ,"'"))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### Fix Star Wars Titles

In [20]:
#Make subset of movies starting with Star Wars
StarWars = Budget_2000[Budget_2000['Movie'].str.startswith('Star Wars')]
StarWars

,Release Date,Movie,Budget,Domestic Gross,Worldwide Gross,Year,Week,Dom_Profits,WW_Profits
6,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,2017,50,303181382,999721747
7,2015-12-18,Star Wars Ep. VII: The Force Awakens,306000000,936662225,2053311220,2015,51,630662225,1747311220
8,2005-05-19,Star Wars Ep. III: Revenge of the Sith,115000000,380270577,848998877,2005,20,265270577,733998877
9,2002-05-16,Star Wars Ep. II: Attack of the Clones,115000000,310676740,656695615,2002,20,195676740,541695615


In [21]:
# Merge so that the Star Wars movies are dropped from df save as new df
Budget_NoStarWars = Budget_2000.merge(StarWars, how='left', indicator=True)
Budget_NoStarWars= Budget_NoStarWars[Budget_NoStarWars['_merge'] == 'left_only']
#make sure the star wars movies are gone
Budget_NoStarWars[Budget_NoStarWars['Movie'].str.startswith('Star Wars')]

,Release Date,Movie,Budget,Domestic Gross,Worldwide Gross,Year,Week,Dom_Profits,WW_Profits,_merge


In [22]:
#edit The Star Wars Movie Title
StarWars['Movie'] = StarWars['Movie'].astype(str).apply(lambda x: x.replace('Ep.','Episode'))
StarWars['Movie'] = StarWars['Movie'].astype(str).apply(lambda x: x.replace(':',' -'))
#make sure movie titles are fixed 
StarWars['Movie']

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


6         Star Wars Episode VIII - The Last Jedi
7      Star Wars Episode VII - The Force Awakens
8    Star Wars Episode III - Revenge of the Sith
9    Star Wars Episode II - Attack of the Clones
Name: Movie, dtype: object

In [23]:
#drop the _merge column that was added 
Budget_NoStarWars.drop(columns=['_merge'], inplace = True)
#concat the formatted starwars subset back to the no starwars df
Budget_2000 = pd.concat([Budget_NoStarWars, StarWars])
#make sure they are there and correct
Budget_2000[Budget_2000['Movie'].str.startswith('Star Wars')]

,Release Date,Movie,Budget,Domestic Gross,Worldwide Gross,Year,Week,Dom_Profits,WW_Profits
6,2017-12-15,Star Wars Episode VIII - The Last Jedi,317000000,620181382,1316721747,2017,50,303181382,999721747
7,2015-12-18,Star Wars Episode VII - The Force Awakens,306000000,936662225,2053311220,2015,51,630662225,1747311220
8,2005-05-19,Star Wars Episode III - Revenge of the Sith,115000000,380270577,848998877,2005,20,265270577,733998877
9,2002-05-16,Star Wars Episode II - Attack of the Clones,115000000,310676740,656695615,2002,20,195676740,541695615


### Formatting Vol to Vol.

In [24]:
#make subset of movies with Vol formating 
GuardGalax = Budget_2000[Budget_2000['Movie'].str.find('Vol ') > -1 ]
GuardGalax

,Release Date,Movie,Budget,Domestic Gross,Worldwide Gross,Year,Week,Dom_Profits,WW_Profits
49,2017-05-05,Guardians of the Galaxy Vol 2,200000000,389813101,862316233,2017,18,189813101,662316233


In [25]:
Budget_GuardGalax = Budget_2000.merge(GuardGalax, how='left', indicator=True)
Budget_GuardGalax= Budget_GuardGalax[Budget_GuardGalax['_merge'] == 'left_only']
Budget_GuardGalax[Budget_GuardGalax['Movie'].str.find('Vol ') > -1 ]

,Release Date,Movie,Budget,Domestic Gross,Worldwide Gross,Year,Week,Dom_Profits,WW_Profits,_merge


In [26]:
GuardGalax['Movie'] = GuardGalax['Movie'].astype(str).apply(lambda x: x.replace('Vol ','Vol. '))
GuardGalax['Movie']

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


49    Guardians of the Galaxy Vol. 2
Name: Movie, dtype: object

In [27]:
Budget_GuardGalax.drop(columns=['_merge'], inplace = True)
Budget_2000 = pd.concat([Budget_GuardGalax, GuardGalax])

In [28]:
Budget_2000[Budget_2000['Movie'].str.find('Vol ') > -1 ]

,Release Date,Movie,Budget,Domestic Gross,Worldwide Gross,Year,Week,Dom_Profits,WW_Profits


### Fix Mission: Impossible gibberish

In [29]:
# Make subset of Mission impossible movies
Mission = Budget_2000[Budget_2000['Movie'].str.startswith('Mission:')]
Mission

,Release Date,Movie,Budget,Domestic Gross,Worldwide Gross,Year,Week,Dom_Profits,WW_Profits
90,2018-07-27,Mission: ImpossibleâFallout,178000000,220159104,787456552,2018,30,42159104,609456552
165,2015-07-31,Mission: ImpossibleâRogue Nation,150000000,195042377,688858992,2015,31,45042377,538858992
177,2006-05-05,Mission: Impossible III,150000000,133501348,397501348,2006,18,-16498652,247501348
195,2011-12-16,Mission: ImpossibleâGhost Protocol,145000000,209397903,694713230,2011,50,64397903,549713230


In [30]:
Budget_NoMission = Budget_2000.merge(Mission, how='left', indicator=True)
Budget_NoMission= Budget_NoMission[Budget_NoMission['_merge'] == 'left_only']
Budget_NoMission[Budget_NoMission['Movie'].str.startswith('Mission:')]

,Release Date,Movie,Budget,Domestic Gross,Worldwide Gross,Year,Week,Dom_Profits,WW_Profits,_merge


In [31]:
Mission['Movie'] = Mission['Movie'].astype(str).apply(lambda x: x.replace('â',' - '))
Mission['Movie']

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


90            Mission: Impossible - Fallout
165      Mission: Impossible - Rogue Nation
177                 Mission: Impossible III
195    Mission: Impossible - Ghost Protocol
Name: Movie, dtype: object

In [32]:
Budget_NoMission.drop(columns=['_merge'], inplace = True)
Budget_2000 = pd.concat([Budget_NoMission, Mission])

In [33]:
Budget_2000[Budget_2000['Movie'].str.startswith('Mission:')]

,Release Date,Movie,Budget,Domestic Gross,Worldwide Gross,Year,Week,Dom_Profits,WW_Profits
90,2018-07-27,Mission: Impossible - Fallout,178000000,220159104,787456552,2018,30,42159104,609456552
165,2015-07-31,Mission: Impossible - Rogue Nation,150000000,195042377,688858992,2015,31,45042377,538858992
177,2006-05-05,Mission: Impossible III,150000000,133501348,397501348,2006,18,-16498652,247501348
195,2011-12-16,Mission: Impossible - Ghost Protocol,145000000,209397903,694713230,2011,50,64397903,549713230


### Double check for â

In [34]:
# Budget_clean[Budget_clean['Movie'].str.find('â')>-1]
Budget_2000['Movie'] = Budget_2000['Movie'].astype(str).apply(lambda x: x.replace('â',' - '))

In [36]:
Budget_2000[Budget_2000['Movie'].str.find('â')>-1]

,Release Date,Movie,Budget,Domestic Gross,Worldwide Gross,Year,Week,Dom_Profits,WW_Profits


### The Twilight Saga , need to be  - 

In [37]:
Twilight = Budget_2000[Budget_2000['Movie'].str.find('Breaking Dawn,')>-1]
Twilight

,Release Date,Movie,Budget,Domestic Gross,Worldwide Gross,Year,Week,Dom_Profits,WW_Profits
218,2012-11-16,"The Twilight Saga: Breaking Dawn, Part 2",136200000,292324737,829724737,2012,46,156124737,693524737
249,2011-11-18,"The Twilight Saga: Breaking Dawn, Part 1",127500000,281287133,689420051,2011,46,153787133,561920051


In [38]:
Budget_2000['Movie'] = Budget_2000['Movie'].astype(str).apply(lambda x: x.replace('Breaking Dawn,','Breaking Dawn -'))

In [39]:
Twilight = Budget_2000[Budget_2000['Movie'].str.find('Breaking Dawn,')>-1]
Twilight

,Release Date,Movie,Budget,Domestic Gross,Worldwide Gross,Year,Week,Dom_Profits,WW_Profits


### Make sure Index is correct in order

In [41]:
Budget_2000.sort_index()

,Release Date,Movie,Budget,Domestic Gross,Worldwide Gross,Year,Week,Dom_Profits,WW_Profits
0,2009-12-18,Avatar,425000000,760507625,2789705275,2009,51,335507625,2364705275
1,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,2011,20,-169536125,635063875
2,2019-04-26,Avengers: Endgame,400000000,856706929,2793009730,2019,17,456706929,2393009730
3,2019-06-07,Dark Phoenix,350000000,65609445,246300941,2019,23,-284390555,-103699059
4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963,2015,18,128405868,1072413963
5,2018-04-27,Avengers: Infinity War,300000000,678815482,2048134200,2018,17,378815482,1748134200
6,2007-05-24,Pirates of the Caribbean: At World's End,300000000,309420425,963420425,2007,21,9420425,663420425
7,2017-11-17,Justice League,300000000,229024295,655945209,2017,46,-70975705,355945209
8,2015-11-06,Spectre,300000000,200074175,879620923,2015,45,-99925825,579620923
9,2012-07-20,The Dark Knight Rises,275000000,448139099,1084439099,2012,29,173139099,809439099


In [42]:
#Save Changes in new df
Budget_clean = Budget_2000

In [43]:
Budget_clean.to_csv('Budget_clean.csv')